In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import function
import pandasql
import sqlite3
from IPython.display import clear_output



In [5]:
df = pd.read_excel('VisitorEntrancesV3.xlsx')

In [6]:
df.head()

,Date,Site_Name,Israelis_Count,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,is_camping,special_activity,...,Season_summer,Season_winter,last_year_visitors,Exceeded_pm10_in_Israel,Site_Exceeded,IsHoliday,IsVacation,IsHeritage,Temperature,isHeatwave
0,2016-03-02,Akhziv,211,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,30.1,0
1,2016-03-17,Akhziv,186,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,19.2,0
2,2016-03-21,Akhziv,59,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,20.1,0
3,2016-03-25,Akhziv,6,0,0,1,0,1,1,0,...,0,0,NaN,1,0,True,True,False,26.5,0
4,2016-04-01,Akhziv,9,0,0,1,0,1,1,0,...,0,0,NaN,0,0,False,True,False,27.7,0


In [7]:
tmp_df = df[['Date','Site_Name','Israelis_Count','Exceeded_pm10_in_Israel']]
totalDays = len(tmp_df.pivot_table(index='Date'))
totalDaysExceeded = tmp_df.pivot_table(index='Date').Exceeded_pm10_in_Israel.sum()

print('total days : ', totalDays)
print('total days exceeded : ', totalDaysExceeded)

total days :  1406
total days exceeded :  62


זיהום בישראל בערים יום לפני לעומת יום אחרי

In [8]:
def testPmSite(df,site):
    dataset = df
    dataset = df.loc[(df.Site_Name==site)&(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(site)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data\n',len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean\n',np.mean(L_EX))
    print('EX median\n',np.median(L_EX))
    print('EX std\n',np.std(L_EX))
    print('EX_Not mean\n',np.mean(l_EX_NOT))
    print('EX_Not median\n',np.median(l_EX_NOT))
    print('EX_Not std\n',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}


pm10SiteTest = pd.DataFrame()
for site in tmp_df.Site_Name.unique():
    pm10SiteTest = pm10SiteTest.append(testPmSite(df,site),ignore_index=True)
clear_output(wait=True)
pm10SiteTest

,Site,Data Size,P_Value,Significant
0,Akhziv,16,0.61,0
1,Amud Stream,24,0.83,0
2,Apollonia,32,0.10,0
3,Arbel,30,0.28,0
4,Ashkelon National Park,29,0.07,0
5,Avdat,31,0.07,0
6,Ayun Stream,31,0.28,0
7,Baram,26,0.13,0
8,Bet Alpha,32,0.97,0
9,Bet Guvrin,32,0.41,0


In [9]:
def testPmAllSite(df):
    dataset = df
    site = 'All Site'
    dataset = df.loc[(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSite(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(46566, 32)
length of paired data 1610 1610
EX mean 239.9304347826087
EX median 90.0
EX std 394.8743162013239
EX_Not mean 273.8776397515528
EX_Not median 110.0
EX_Not std 449.8688469297552
pval -  0.00000038 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_16848\3687236785.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSite(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,All Site,1610,0.0,1


In [10]:
def testPmAllSiteNorth(df):
    dataset = df
    site = 'region_North'
    dataset = df.loc[(df.region_North==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteNorth(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(26998, 32)
length of paired data 923 923
EX mean 210.30227518959913
EX median 80.0
EX std 342.3583410066057
EX_Not mean 228.53196099674972
EX_Not median 89.0
EX_Not std 379.58118250767205
pval -  0.00423329 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_16848\3914921657.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteNorth(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_North,923,0.0,1


In [11]:
def testPmAllSiteSouth(df):
    dataset = df
    site = 'region_South Site'
    dataset = df.loc[(df.region_South==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteSouth(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(9438, 32)
length of paired data 336 336
EX mean 338.8958333333333
EX median 119.5
EX std 524.8635121736035
EX_Not mean 383.0922619047619
EX_Not median 144.5
EX_Not std 575.6938467434032
pval -  0.00046304 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_16848\3451315854.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteSouth(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_South Site,336,0.0,1


In [12]:
def testPmAllSiteCenter(df):
    dataset = df
    site = 'region_Central Site'
    dataset = df.loc[(df.region_Central==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteCenter(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(5095, 32)
length of paired data 176 176
EX mean 326.4318181818182
EX median 174.0
EX std 456.87526384657343
EX_Not mean 437.02840909090907
EX_Not median 180.0
EX_Not std 574.7411317376767
pval -  0.01199240 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_16848\2640737381.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteCenter(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_Central Site,176,0.01,1


In [ ]:
def testPmAllSite_Judea_Samaria(df):
    dataset = df
    site = 'region_Judea_Samaria Site'
    dataset = df.loc[(df.region_Judea_Samaria==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Exceeded_pm10_in_Israel'][i-1]
        now = tmp['Exceeded_pm10_in_Israel'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSite_Judea_Samaria(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

In [ ]:
df

In [ ]:
tmp_df = df[['Date','Site_Name','Israelis_Count','Exceeded_pm10_in_Israel','Site_Exceeded']]


In [ ]:
def testPmAllSite_inExcInSite(df):
    dataset = df
    site = 'All Site'
    dataset = df.loc[(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==1 and now== 0 :
            L_EX.append(tmp['Israelis_Count'][i-1])
            l_EX_NOT.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}


pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSite_inExcInSite(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest